In [1]:
# example of training an unstable gan for generating a handwritten digit
from os import makedirs
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.initializers import RandomNormal
from matplotlib import pyplot

# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # define model
    model = Sequential()
    # downsample to 14x14
    model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init,
    input_shape=in_shape))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    # downsample to 7x7
    model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    # classifier
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# define the standalone generator model
def define_generator(latent_dim):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # define model
    model = Sequential()
    # foundation for 7x7 image
    n_nodes = 128 * 7 * 7
    model.add(Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 128)))
    # upsample to 14x14
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',
    kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 28x28
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',
    kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    # output 28x28x1
    model.add(Conv2D(1, (7,7), activation='tanh', padding='same', kernel_initializer=init))
    return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

# load mnist images
def load_real_samples():
    # load dataset
    (trainX, trainy), (_, _) = load_data()
    # expand to 3d, e.g. add channels
    X = expand_dims(trainX, axis=-1)
    # select all of the examples for a given class
    selected_ix = trainy == 8
    X = X[selected_ix]
    # convert from ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    return X

# # select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # select images
    X = dataset[ix]
    # generate class labels
    y = ones((n_samples, 1))
    return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = generator.predict(x_input)
    # create class labels
    y = zeros((n_samples, 1))
    return X, y

# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, latent_dim, n_samples=100):
    # prepare fake examples
    X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
    # scale from [-1,1] to [0,1]
    X = (X + 1) / 2.0
    # plot images
    for i in range(10 * 10):
        # define subplot
        pyplot.subplot(10, 10, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
    # save plot to file
    pyplot.savefig('results_collapse/generated_plot_%03d.png' % (step+1))
    pyplot.close()
    # save the generator model
    g_model.save('results_collapse/model_%03d.h5' % (step+1))
    
# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist):
    # plot loss
    pyplot.subplot(2, 1, 1)
    pyplot.plot(d1_hist, label='d-real')
    pyplot.plot(d2_hist, label='d-fake')
    pyplot.plot(g_hist, label='gen')
    pyplot.legend()
    # plot discriminator accuracy
    pyplot.subplot(2, 1, 2)
    pyplot.plot(a1_hist, label='acc-real')
    pyplot.plot(a2_hist, label='acc-fake')
    pyplot.legend()
    # save plot to file
    pyplot.savefig('results_collapse/plot_line_plot_loss.png')
    pyplot.close()
    
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=128):
    # calculate the number of batches per epoch
    bat_per_epo = int(dataset.shape[0] / n_batch)
    # calculate the total iterations based on batch and epoch
    n_steps = bat_per_epo * n_epochs
    # calculate the number of samples in half a batch
    half_batch = int(n_batch / 2)
    # prepare lists for storing stats each iteration
    d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()
    # manually enumerate epochs
    for i in range(n_steps):
        # get randomly selected ✬real✬ samples
        X_real, y_real = generate_real_samples(dataset, half_batch)
        # update discriminator model weights
        d_loss1, d_acc1 = d_model.train_on_batch(X_real, y_real)
        # generate ✬fake✬ examples
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        # update discriminator model weights
        d_loss2, d_acc2 = d_model.train_on_batch(X_fake, y_fake)
        # prepare points in latent space as input for the generator
        X_gan = generate_latent_points(latent_dim, n_batch)
        # create inverted labels for the fake samples
        y_gan = ones((n_batch, 1))
        # update the generator via the discriminator✬s error
        for j in range(2): # Generator updated twice per each discriminator update (to avoid convergence failure)
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
        # summarize loss on this batch
        print('>%d, d1=%.3f, d2=%.3f g=%.3f, a1=%d, a2=%d' %
                    (i+1, d_loss1, d_loss2, g_loss, int(100*d_acc1), int(100*d_acc2)))
        # record history
        d1_hist.append(d_loss1)
        d2_hist.append(d_loss2)
        g_hist.append(g_loss)
        a1_hist.append(d_acc1)
        a2_hist.append(d_acc2)
        # evaluate the model performance every ✬epoch✬
        if (i+1) % bat_per_epo == 0:
            summarize_performance(i, g_model, latent_dim)
    plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist)

# make folder for results
makedirs('results_collapse', exist_ok=True)
# size of the latent space
latent_dim = 1
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = load_real_samples()
print(dataset.shape)
# train model
train(generator, discriminator, gan_model, dataset, latent_dim)

Using TensorFlow backend.


(5851, 28, 28, 1)


C:\Users\Admin\anaconda3\envs\gputest\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\Users\Admin\anaconda3\envs\gputest\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, d1=0.820, d2=0.708 g=0.056, a1=48, a2=18


C:\Users\Admin\anaconda3\envs\gputest\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>2, d1=0.138, d2=0.926 g=0.010, a1=100, a2=0
>3, d1=0.060, d2=0.930 g=0.003, a1=100, a2=0
>4, d1=0.044, d2=0.968 g=0.002, a1=100, a2=0
>5, d1=0.029, d2=0.894 g=0.002, a1=100, a2=0
>6, d1=0.029, d2=0.881 g=0.003, a1=100, a2=0
>7, d1=0.021, d2=0.846 g=0.003, a1=100, a2=0
>8, d1=0.023, d2=0.781 g=0.004, a1=100, a2=0
>9, d1=0.020, d2=0.745 g=0.003, a1=100, a2=17
>10, d1=0.016, d2=0.738 g=0.003, a1=100, a2=23
>11, d1=0.015, d2=0.718 g=0.003, a1=100, a2=35
>12, d1=0.016, d2=0.696 g=0.002, a1=100, a2=56
>13, d1=0.019, d2=0.694 g=0.003, a1=100, a2=51
>14, d1=0.017, d2=0.671 g=0.002, a1=100, a2=87
>15, d1=0.012, d2=0.657 g=0.002, a1=100, a2=98
>16, d1=0.009, d2=0.645 g=0.002, a1=100, a2=100
>17, d1=0.009, d2=0.626 g=0.003, a1=100, a2=100
>18, d1=0.009, d2=0.620 g=0.002, a1=100, a2=100
>19, d1=0.007, d2=0.605 g=0.002, a1=100, a2=100
>20, d1=0.008, d2=0.585 g=0.001, a1=100, a2=100
>21, d1=0.016, d2=0.565 g=0.002, a1=100, a2=100
>22, d1=0.012, d2=0.538 g=0.002, a1=100, a2=100
>23, d1=0.012, d2=0.5

>172, d1=0.002, d2=0.005 g=0.001, a1=100, a2=100
>173, d1=0.002, d2=0.003 g=0.001, a1=100, a2=100
>174, d1=0.003, d2=0.004 g=0.001, a1=100, a2=100
>175, d1=0.004, d2=0.007 g=0.001, a1=100, a2=100
>176, d1=0.003, d2=0.004 g=0.001, a1=100, a2=100
>177, d1=0.002, d2=0.005 g=0.001, a1=100, a2=100
>178, d1=0.002, d2=0.006 g=0.001, a1=100, a2=100
>179, d1=0.004, d2=0.006 g=0.001, a1=100, a2=100
>180, d1=0.003, d2=0.006 g=0.001, a1=100, a2=100
>181, d1=0.004, d2=0.006 g=0.001, a1=100, a2=100
>182, d1=0.004, d2=0.008 g=0.001, a1=100, a2=100
>183, d1=0.003, d2=0.006 g=0.002, a1=100, a2=100
>184, d1=0.006, d2=0.014 g=0.002, a1=100, a2=100
>185, d1=0.004, d2=0.004 g=0.001, a1=100, a2=100
>186, d1=0.005, d2=0.013 g=0.001, a1=100, a2=100
>187, d1=0.008, d2=0.006 g=0.001, a1=100, a2=100
>188, d1=0.004, d2=0.006 g=0.001, a1=100, a2=100
>189, d1=0.003, d2=0.003 g=0.001, a1=100, a2=100
>190, d1=0.002, d2=0.003 g=0.001, a1=100, a2=100
>191, d1=0.002, d2=0.009 g=0.001, a1=100, a2=100
>192, d1=0.007, d2=0

>343, d1=0.559, d2=0.510 g=2.164, a1=71, a2=70
>344, d1=0.601, d2=0.317 g=2.080, a1=70, a2=100
>345, d1=0.524, d2=0.400 g=2.106, a1=73, a2=84
>346, d1=0.268, d2=0.127 g=1.704, a1=90, a2=100
>347, d1=0.300, d2=0.217 g=1.000, a1=92, a2=100
>348, d1=0.209, d2=0.265 g=0.963, a1=92, a2=100
>349, d1=0.312, d2=0.278 g=1.076, a1=89, a2=100
>350, d1=0.441, d2=0.289 g=0.898, a1=76, a2=100
>351, d1=0.287, d2=0.249 g=0.816, a1=84, a2=100
>352, d1=0.326, d2=0.328 g=0.931, a1=89, a2=100
>353, d1=0.255, d2=0.216 g=0.944, a1=92, a2=100
>354, d1=0.421, d2=0.387 g=0.989, a1=78, a2=100
>355, d1=0.310, d2=0.339 g=1.381, a1=87, a2=100
>356, d1=0.540, d2=0.379 g=1.536, a1=73, a2=100
>357, d1=0.391, d2=0.250 g=1.650, a1=78, a2=100
>358, d1=0.373, d2=0.303 g=1.791, a1=81, a2=100
>359, d1=0.428, d2=0.345 g=2.203, a1=81, a2=100
>360, d1=0.367, d2=0.168 g=2.003, a1=84, a2=100
>361, d1=0.295, d2=0.220 g=2.074, a1=90, a2=100
>362, d1=0.172, d2=0.143 g=2.406, a1=95, a2=100
>363, d1=0.363, d2=0.211 g=2.156, a1=85, a